Download overlay and grab DMA aliases

<div class="alert alert-box alert-warning">
For <b>Kria</b> boards use the `Debug Bridge` IP instead of the JTAG cable.
</div>

In [ ]:
from pynq import Overlay, allocate, PL
import numpy as np

PL.reset()

ol = Overlay('dma_hp32.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

In Vivado open **Open Hardware Manger**, then **Open Target** then **Auto Connect**

Allocate intput and output buffer, then initialize input buffer

In [ ]:
data_size = 20

input_buffer = allocate(shape=(data_size,), dtype=np.uint32)
output_buffer = allocate(shape=(data_size,), dtype=np.uint32)

input_buffer[:] = np.arange(data_size, data_size+data_size, dtype=np.uint32)

In Vivado Hardware Manager, set the trigger to start capturing when `TVALID` of *axi_dma_M_AXIS_MM2S* is 1 or in the rising edge.

Set the number of windows to 1 with a depth of 1024 and the trigger position to 0

In [ ]:
dma_recv.transfer(output_buffer)
dma_send.transfer(input_buffer)
dma_send.wait()
dma_recv.wait()

For Zynq devices and MPSoC (only when width is 32), you should see that elements in the array in the odd index are zero.

Compare input and output buffers

In [ ]:
print(f'{input_buffer=}')
print(f'{output_buffer=}')

In [ ]:
if ol.ip_dict.get('processing_system7_0'):
    ps = ol.ip_dict.get('processing_system7_0')
    hp_ports = [f'C_S_AXI_HP{idx}_DATA_WIDTH' for idx in range(4)]
    width = [64]
else:
    ps = ol.ip_dict.get('zynq_ultra_ps_e')
    hp_ports = [f'C_SAXIGP{idx}_DATA_WIDTH' for idx in range(6)]
    width = [128, 64]

for i in hp_ports:
    w = ps['parameters'][i]
    print(f"{i}: {w}, configured correctly? {int(w) in width}")

Release the buffers and overlay

In [ ]:
del input_buffer, output_buffer
ol.free()